# End of week 1 exercise (with user input(question, model)

In [0]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [0]:
# set up environment and constants
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

In [0]:
system_prompt = "You are an expert coder with educational skills for beginners. \
You are able to explain, debbug or generate code in Python, R or bash, and to provide examples of use case if applicable. \
Please add references to relevant sources if available. If not, do not invent.\n"
system_prompt += "this is an example of a response:"
system_prompt += """
Sure! Here’s the explanation in plain text format, suitable for Markdown:

# Explanation of the Code

### Code:
```python
full_name = lambda first, last: f'Full name: {first.title()} {last.title()}'
```

### Explanation:

1. **Lambda Function:**
   - The keyword `lambda` is used to create a small, one-line anonymous function (a function without a name).
   - It takes two parameters: `first` (for the first name) and `last` (for the last name).

2. **String Formatting (`f-string`):**
   - `f'Full name: {first.title()} {last.title()}'` is a formatted string (f-string).
   - It inserts the values of `first` and `last` into the string while applying `.title()` to capitalize the first letter of each name.

3. **Assigning the Function:**
   - The lambda function is assigned to the variable `full_name`, so we can use `full_name()` like a regular function.

### How to Use It:
Now, let’s call this function and see what it does.

```python
print(full_name("john", "doe"))
```

#### Output:
```
Full name: John Doe
```

### What Happens:
- `"john"` becomes `"John"` (because `.title()` capitalizes the first letter).
- `"doe"` becomes `"Doe"`.
- The output is `"Full name: John Doe"`.

### Summary:
This is a simple way to create a function that formats a full name while ensuring proper capitalization. You could write the same function using `def` like this:

```python
def full_name(first, last):
    return f'Full name: {first.title()} {last.title()}'
```

Both versions work the same way, but the `lambda` version is more compact.

### Reference(s):
To deepen your understanding of the code snippet involving Python's lambda functions here is a resource you might find helpful:

Ref. **Python Lambda Functions:**
   - The official Python documentation provides an in-depth explanation of lambda expressions, including their syntax and use cases.
     - [Lambda Expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions)

```
You can copy and paste this into any Markdown file or viewer. Let me know if you need further modifications! 😊
"""

In [0]:
# display(Markdown(system_prompt))

In [0]:
# user question
default_question= """
Please explain what this code does and why:
yield from {book.get('author') from book in books if book.get('author')}
"""
user_question= str(input("What code do you want me to explain?/n(Press 'Enter' for an example)"))

if user_question== '':
    question= default_question
    print(default_question)
else:
    question= "Please explain what this code does and why:\n" + user_question

In [0]:
print(question)

In [0]:
# user model
model_number= input("""
Please enter the number of the model you want to use from the list below:
1 GPT-4o Mini
2 Llama 3.2
3 DeepSeek R1
4 Qwen 2.5
""")
try:
    if int(model_number)==1:
        model= 'gpt-4o-mini'
    elif int(model_number)==2:
        model= 'llama3.2'
    elif int(model_number)==3:
        model= 'deepseek-r1:1.5b'
    elif int(model_number)==4:
        model= 'qwen2.5:3b'
    else:
        model= ''
        print("please provide only a number from the list")
except:
    model=''
    print("Please provide a number or press 'Enter' to finish")

In [0]:
# print(model)

In [0]:
messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
    ]

In [0]:
if int(model_number)==1:
    openai= OpenAI()
    stream = openai.chat.completions.create(
        model=model,
        messages=messages,
        stream= True
        )

    response = ""
    print("The following answer will be generated by {0} LLM".format(model))
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
elif int(model_number)==2 or 3 or 4:
    !ollama pull {model}
    print("\n\nThe following answer will be generated by {0} LLM\n\n".format(model))
    response = ollama.chat(
        model=model,
        messages = messages)
    result= response['message']['content']
    display(Markdown(result))